# Proyecto Deep Learning

Profesor:

Integrantes:
- Miguel Castillo Huebner
- Pedro Betanzo J.
- Fernando Basterrechea
- Ricardo Muñoz Ortega

## Reconocimiento de señales de tránsito

El reconocimiento automatizado de señales de tránsito es fundamental para mejorar la seguridad vial y habilitar tecnologías de conducción autónoma. Los sistemas avanzados de asistencia al conductor dependen de una clasificación precisa de las señales para interpretar el entorno vial en tiempo real. Este proyecto busca clasificar 58 categorías de señales de tránsito utilizando Deep Learning, con un conjunto de entrenamiento de aproximadamente 120 imágenes por clase y un conjunto de prueba con 1994 imágenes. 

## Marco Teórico

Las arquitecturas de redes neuronales mas conocidas son:

1. **Perceptrón Multicapa (MLP)**: Red neuronal donde cada capa esta conectada a todas las neuronas de la capa siguiente. Se utiliza para tareas básicas de clasificación y regresión. Datos estructurados y reducidos.

2. **Redes Neuronales Convolucionales (CNN)**: Utilizan capas de convolución que aplican filtros sobre las entradas para extraer características espaciales, especialmente de imágenes. Se emplean en el procesamiento de imágenes, clasificación, detección de objetos, segmentación, reconocimiento facial y videos.

3. **Redes Neuronales Recurrentes (RNN)**: Redes con conexiones de retroalimentación, cuentan con un estado y procesa secuencia de datos. Se utilizan en NLP, series temporales, traducción de automática, análisis de sentimiento y generación de texto.

4. **Redes de Memoria a Largo Plazo (LSTM)**: Tiene la capacidad de recordar información a largo plazo, soluciona el problema del desvanecimiento del gradiente. se emplea en series temporales, predicción financiera, generación de texto, traducción, procesamiento de voz y video secuencial.

5. **Unidades de Puerta Recurrente (GRU)**: Estructura mas simple y con menos parámetros de una LSTM. Se emplean en problemas de secuencias.

6. **Redes Generativas Antagónicas (GAN)**: Se compone de un red generadora y otra desciminadora que compiten, la funcion de la red generadora es crear datos falsos que el discriminador debe clasificar. Se emplea en generación de imágenes, superresolución, generación de datos, deepfake y arte generativo. 

7. **Autoencoders**: Redes que comprimen los datos de entrada en una representación de menor dimensión para luego reconstruirlos. Se emplea en compresión de datos, reducción de dimensionalidad, eliminación de ruido, generación de datos y detección de anomalías.

8. **Redes de Convolución 3D (3D-CNN)**: Capturar información espacial y temporal, su funcionamiento es similar a una CNN. Se utiliza en análisis de video, diagnóstico médico, detección de acciones y procesamiento de secuencia de imágenes.

9. **Transformers**: Cuentan con mecanismos de autoatención para modelar relaciones de largo alcance en secuencias, permiten el procesamiento en paralelo de mejor forma que las RNN. Utilizadas en traducción automática, generación de texto (GPT, BERT), chatbots, análisis de sentimientos, modelación de secuencias y visión por computadoras. 

10. **Redes de Memoria Diferenciable (DNC)**: Utilizan memorias externas, permitiendo labores mas complejas. Empleados en navegación en grafos, recuperación de información y procesamiento de datos con estructuras complejas.

11. **Redes de Capsulas (CapsNet)**: Capsulas que capturan relaciones espaciales de las características. Utilizadas en reconocimiento de imagenes, detección de objetos 3D y visión por computadora.

12. **Redes de Refuerzo Profundo**: Combinación de las arquitecturas anteriores con aprendizaje por refuerzo, permitiendo al agente aprender a tomar desiciones. Empleadas en juegos, robótica, sistemas de recomendación, autónomia y simulaciones.



## Dependencias

In [ ]:
import torch
from torchvision import datasets, transforms
import pandas as pd
import os
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image

## Dataset

Existen 58 clases y 120 imagenes por cada una. El archivo <code>labels.csv</code> contiene la descripción de las clases de las señales de tránsito. 

Dispobible en: [dataset](https://www.kaggle.com/datasets/ahemateja19bec1025/traffic-sign-dataset-classification/data)


In [3]:
etiquetas = pd.read_csv('labels.csv')
etiquetas.sample(3)

,ClassId,Name
24,24,Go Right
39,39,Dangerous curve to the right
52,52,Unknown6


In [51]:
etiquetas.shape

(58, 2)

In [11]:
ruta = 'traffic_Data'
i = 0
j = 0
for carpeta, subcarpeta, archvos in os.walk(ruta):
    nivel = carpeta.replace(ruta, '').count(os.sep)
    identacion = ' ' * 4 * nivel
    print(f'{identacion}{os.path.basename(carpeta)}')
    subidentacion = ' ' * 4 * (nivel+1)
    for archivo in archvos:
        print(f'{subidentacion}{archivo}')
        if j == 2:
            break
        j+=1
    j = 0
    if i == 3:
        break
    i+=1

traffic_Data
    DATA
        0
            000_0001.png
            000_0002.png
            000_0003.png
        1
            001_0001.png
            001_0002.png
            001_0003.png


In [12]:
etiquetas.head(3)

,ClassId,Name
0,0,Speed limit (5km/h)
1,1,Speed limit (15km/h)
2,2,Speed limit (30km/h)


## Preprocesamiento

- 128x128 o 224x224: Balance entre detalle y eficiencia, común para tareas de clasificación en modelos como ResNet y MobileNet.

- 256x256 o 512x512: Útil para tareas que requieren más detalles (como detección de objetos compleja).

In [37]:
transformacion = transforms.Compose(
    [
        transforms.Resize((64,64)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, ),(0.5, ))
    ]
)

dataset = datasets.ImageFolder(root='traffic_data/DATA',transform=transformacion)

## División de los datos

In [38]:
n_train =  int(0.8 * len(dataset))
train, val = random_split(
    dataset,
    [
        n_train,
        len(dataset) - n_train
    ]
)

In [39]:
cargador_entr = DataLoader(
    train,
    batch_size=32,
    shuffle=True
) 
cargador_val = DataLoader(
    val,
    batch_size=32,
    shuffle=False
)

## Ajuste Modelo

In [40]:
class SeñalesRed(nn.Module):

    def __init__(self, num_clasess=58):
        super(SeñalesRed, self).__init__()
        self.conv1 = nn.Conv2d(
            3,
            32,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.conv2 = nn.Conv2d(
            32,
            64,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.conv3 = nn.Conv2d(
            64,
            128,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.fc1 = nn.Linear(128*8*8,512)
        self.fc2 = nn.Linear(512, num_clasess)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [41]:
modelo_cnn = SeñalesRed(num_clasess=58) 

## Optimizador y Función de Pérdida

In [42]:
floss = nn.CrossEntropyLoss() 
opt = optim.Adam(modelo_cnn.parameters(), lr=0.001)

## Ajuste Modelo

In [43]:
num_epochs = 10

for epoch in range(num_epochs):
    modelo_cnn.train()
    running_loss = 0.0
    for images, labels in cargador_entr:
        opt.zero_grad()
        outputs = modelo_cnn(images)
        loss = floss(outputs, labels)
        loss.backward()
        opt.step()
        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(cargador_entr)}")

    # Evaluación en el conjunto de validación
    modelo_cnn.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in cargador_val:
            outputs = modelo_cnn(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy: {100 * correct / total}%")

    torch.save(modelo_cnn.state_dict(),'SeñalesRed.pth')

Epoch 1/10, Loss: 2.239330992812202
Accuracy: 70.14388489208633%
Epoch 2/10, Loss: 0.618167235028176
Accuracy: 88.48920863309353%
Epoch 3/10, Loss: 0.23243463266463507
Accuracy: 93.76498800959233%
Epoch 4/10, Loss: 0.1294415474492347
Accuracy: 96.76258992805755%
Epoch 5/10, Loss: 0.05143349222018428
Accuracy: 97.60191846522781%
Epoch 6/10, Loss: 0.03274121239900567
Accuracy: 97.84172661870504%
Epoch 7/10, Loss: 0.029865282966472607
Accuracy: 97.84172661870504%
Epoch 8/10, Loss: 0.022681608736110363
Accuracy: 98.6810551558753%
Epoch 9/10, Loss: 0.024654920358720814
Accuracy: 97.96163069544365%
Epoch 10/10, Loss: 0.0069416887126863
Accuracy: 98.8009592326139%


## Testing modelo

In [57]:
ruta_img = 'traffic_Data/TEST/055_0021_j.png'
imagen = Image.open(ruta_img).convert('RGB')
imagen = transformacion(imagen)
imagen = imagen.unsqueeze(0)


In [58]:
with torch.no_grad():
    salida = modelo_cnn(imagen)

_, prediccion = torch.max(salida, 1)  
clase_pred = prediccion.item() 
print("Clase:", etiquetas[etiquetas['ClassId']==clase_pred] )

Clase:     ClassId                     Name
51       51  Heavy Vehicle Accidents


In [56]:
etiquetas[etiquetas['ClassId']==55]

,ClassId,Name
55,55,No entry
